# Projeto 1 - Ciência dos Dados

Nome:Livia Brigido do Nascimento

Nome:Fabio Dotoli Ferreira Filho

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [28]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

In [29]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/fabiodff/Desktop/P1Cdados2021_2


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [30]:
filename = 'Airpods.xlsx'

In [31]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Treinamento,Classificacao_Treino
0,alguem me da um apple watch 😭😭😭😭 um macbook pr...,0
1,@hschmaiske @matheus1lva airpods pro é bom? é....,1
2,mais uns airpods estragados… que sorte 🤦🏻‍♂️,1
3,"@annaeso @masbahnana isso eu sei, mas nem espe...",0
4,sonhando com o dia que vou achar airpods por m...,1


In [32]:
train.Classificacao_Treino.value_counts(True)

0    0.72
1    0.28
Name: Classificacao_Treino, dtype: float64

In [33]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificacao_Teste
0,"favela venceu, o pai lançou um airpods monstro...",0
1,"[weverse]\n\n“namjoon oppa, se eles lançarem a...",0
2,@priboechat comprei um fone tipo airpods e amei,0
3,@_army_brasileira_7 \n _army_brasileira_7 \n\n...,0
4,"@abelardo_moura ainda bem que foi o de fio, po...",0


In [34]:
test.Classificacao_Teste.value_counts(True)

0    0.713333
1    0.286667
Name: Classificacao_Teste, dtype: float64

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

### Produto escolhido ###
Os Airpods, lançamento da Apple no ano de 2016, são pequenos fones com conexão via bluetooth, que vêm acompanhados do estojo (usado para recarregá-los) e de um cabo lightning (carregador).

- Os tweets considerados relevantes foram aqueles que citavam alguma caracteristica relacionada ao produto escolhido, como por exemplo a durabilidade, a facilidade em perder os fones, preço e críticas/elogios aos Airpods como um todo.

- Os tweets irrelevantes, por sua vez, são aqueles que citam o produto, porém de uma forma que não é possivel extrair um feedback significativo sobre satisfação ou qualidades (positivas ou negativas) como, por exemplo, uma frase citando alguns itens aleatórios (dentre eles, os Airpods).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [93]:
#Montando a função de limpeza principal e para a limpeza de palavras mais específicas** (cleanup)

import re #utilizaremos para limpar o texto ('replace')



def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text) #substitui o primeiro argumento pelo segundo, e o terceiro argumento é onde será aplicada a substituição
    text_subbed = re.sub('\n', '', text_subbed) #retirando o enter (pular para a próxima linha)
    
    padrao = r'https\w+'
    pattern2 = re.compile(padrao)
    text_subbed2 = re.sub(pattern2, '', text_subbed)
    
    return text_subbed2.lower()


def cleanup_links(text):
    
    padrao1 = r'@\w+' #remove o símbolo ou escrita e as letras após ele
    pattern1 = re.compile(padrao1)
    text_subbed = re.sub(pattern1, '', text)  
    
    padrao3 = r'#\w+'
    pattern3 = re.compile(padrao3)
    text_subbed3 = re.sub(pattern3, '', text_subbed)
    
    padrao4 = r'&\w+'
    pattern4 = re.compile(padrao4)
    text_subbed4 = re.sub(pattern4, '', text_subbed3)
    
    
    return text_subbed4
    

In [94]:
#Célula de Testes das funções:
variavel_teste="@joniel,  gosta! do: https:'hgah, a gente #skaska &sasjas"

variavel_limpa = cleanup_links(variavel_teste)
variavel_limpa
variavel_limpa2= cleanup(variavel_limpa)
variavel_limpa2

'  gosta do  a gente  '

### Para os tweets Relevantes: ###

In [50]:
#perguntar como retirar os @ e oa nomes de usuários que os acompanham
#perguntar sobre os espaços entre palavras e emojis
#pesquisar sobre a suavização de laplace

In [95]:
#já foi dividido em rel. e irrel. pois depois não será possivel dividir por ser um df de apenas uma coluna 
train_relevantes_clean = train.loc[train["Classificacao_Treino"] == 1, 'Treinamento'].apply(cleanup_links)
train_relevantes_clean.apply(cleanup).to_frame()


,Treinamento
1,airpods pro é bom é qualidade boa sim noise ...
2,mais uns airpods estragados… que sorte 🤦🏻‍♂️
4,sonhando com o dia que vou achar airpods por m...
5,tava pensando se esse airpods tinha valido o p...
7,oh trenzin q doi os zovido é esses airpods
...,...
583,passando 3 meses encontrei os meus airpods
591,esqueci de carregar o airpods treinar sem músi...
593,perdi meus airpods 🥲
594,airpods compramos 2 um pra mim um pra minh...


In [96]:
#train_rel_clean.astype(str)
train_relevantes_lower = train_relevantes_clean.str.lower() #por que usar o .str ao inves de uma variavel-- Não quero trabalhar no dataset,mas em cada linha.
train_relevantes = train_relevantes_lower.str.split()


In [97]:
type(train_relevantes_clean)

pandas.core.series.Series

In [98]:
train_relevantes_l = []  #criando uma lista para englobar todas as outras listas criadas com o .split

for palavras in train_relevantes:
    train_relevantes_l += palavras
serie_train_relevantes = pd.Series(train_relevantes_l)
serie_train_relevantes

0         airpods
1             pro
2               é
3            bom?
4              é.
          ...    
2442          uma
2443          boa
2444    ideia????
2445            q
2446      inferno
Length: 2447, dtype: object

In [99]:
serie_train_relevantes_absoluta = serie_train_relevantes.value_counts(True)
serie_train_relevantes_absoluta

airpods    0.060482
o          0.028198
de         0.026154
que        0.022477
um         0.020842
             ...   
psé,       0.000409
vei        0.000409
ensaio     0.000409
doi        0.000409
questão    0.000409
Length: 945, dtype: float64

### Para os tweets Irrelevantes: ###

In [105]:
train_irrelevantes_clean = train.loc[train["Classificacao_Treino"] == 0, 'Treinamento'].apply(cleanup_links)
train_irrelevantes_clean.apply(cleanup).to_frame()


,Treinamento
0,alguem me da um apple watch 😭😭😭😭 um macbook pr...
3,isso eu sei mas nem esperaria algo assim é q...
6,sete anos atrás em 09/09/2014 o valor do dóla...
8,queria tanto um fone airpods mas zero dinheiro
9,necessito de um airpods
...,...
592,items do dreamcatcher para uma boa noite de so...
595,vale a pena pegar um airpods pro
597,me compra um airpods
598,cara de mau mas no airpods tá tocando harry st...


In [101]:
train_irrelevantes_lower = train_irrelevantes_clean.str.lower() 
train_irrelevantes = train_irrelevantes_lower.str.split()

In [102]:
type(train_irrelevantes_clean)

pandas.core.series.Series

In [103]:
train_irrelevantes_l = []  #criando uma lista para englobar todas as outras listas criadas com o .split

for palavras in train_irrelevantes:
    train_irrelevantes_l += palavras
serie_train_irrelevantes = pd.Series(train_irrelevantes_l)
serie_train_irrelevantes

0          alguem
1              me
2              da
3              um
4           apple
          ...    
7512            o
7513       iphone
7514    reconhece
7515         como
7516     original
Length: 7517, dtype: object

In [104]:
serie_train_irrelevantes_absoluta = serie_train_irrelevantes.value_counts(True)
serie_train_irrelevantes_absoluta

airpods    0.057337
o          0.025542
de         0.024345
e          0.024212
um         0.023946
             ...   
live       0.000133
falaria    0.000133
insetos    0.000133
smart      0.000133
frete      0.000133
Length: 2136, dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**